# 使用 MCP 的工具 ⏰

模型上下文协议（MCP）是一种开放协议，规范了应用程序如何向LLM提供工具和上下文。LangChain代理可以使用MCP服务器上定义的工具，使用`langchain-mcp-adapters`库。

langchain-mcp-adapters 使代理能够使用跨越一个或多个 MCP 服务器定义的工具。

<img src="./assets/mcp.png" align="left" width="700">

## 初始化
加载并检查所需的环境变量

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 文件加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env(".env")

DASHSCOPE_API_KEY=****931f
DASHSCOPE_BASE_URL=****e/v1
LANGSMITH_API_KEY=****ef8f
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****s-56
LANGSMITH_ENDPOINT=****.com


In [5]:
import sys
print(hasattr(sys.stderr, "fileno"))
# Notebook 上通常为 False 或会在调用 fileno() 时抛异常

True


In [9]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# 连接 mcp-time 服务器以进行时区相关操作
# 该 Go 服务器提供当前时间、相对时间解析、
# 时区转换、时长计算与时间比较等工具
mcp_client = MultiServerMCPClient(
    {
        "time": {
      "transport": "streamable_http",
      "url": "https://mcp.api-inference.modelscope.net/adcc1ca6e6d642/mcp"
    }
    },
)

# 从 MCP 服务器加载工具
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

Loaded 2 MCP tools: ['get_current_time', 'convert_time']


使用 MCP 提供的时间工具创建一个代理。

In [10]:
from langchain.agents import create_agent
from langchain_qwq import ChatQwen
import os
llm=ChatQwen(
    model="qwen3-max",
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY")
)
agent_with_mcp = create_agent(
    model=llm,
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

询问旧金山的当前时间。

In [11]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the time in SF right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What's the time in SF right now?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_719ae124b2b242fd8648fb27)
 Call ID: call_719ae124b2b242fd8648fb27
  Args:
    timezone: America/Los_Angeles
================================= Tool Message =================================
Name: get_current_time

{
  "timezone": "America/Los_Angeles",
  "datetime": "2025-12-03T02:27:23-08:00",
  "day_of_week": "Wednesday",
  "is_dst": false
}
================================== Ai Message ==================================

The current time in San Francisco is 2:27 AM on Wednesday, December 3, 2025.
